In [1]:
using ParallelTemperingMonteCarlo
using BenchmarkTools,DelimitedFiles,StaticArrays,Random

By the end of this module, the ptmc_run function will be complete. First we aim to split the dependencies from MCRun, then I aim to test variable function definitions.

In [2]:
Random.seed!(1234)

# number of atoms
n_atoms = 13

# temperature grid
ti = 5.
tf = 16.
n_traj = 32

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details


mc_cycles = 400000 #default 20% equilibration cycles on top


mc_sample = 1  #sample every mc_sample MC cycles

#move_atom=AtomMove(n_atoms) #move strategy (here only atom moves, n_atoms per MC cycle)
displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)

#ELJpotential for neon
#c1=[-10.5097942564988, 0., 989.725135614556, 0., -101383.865938807, 0., 3918846.12841668, 0., -56234083.4334278, 0., 288738837.441765]
#elj_ne1 = ELJPotential{11}(c1)

c=[-10.5097942564988, 989.725135614556, -101383.865938807, 3918846.12841668, -56234083.4334278, 288738837.441765]
pot = ELJPotentialEven{6}(c)

#starting configurations
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
pos_ne13 = [[2.825384495892464, 0.928562467914040, 0.505520149314310],
[2.023342172678102,	-2.136126268595355, 0.666071287554958],
[2.033761811732818,	-0.643989413759464, -2.133000349161121],
[0.979777205108572,	2.312002562803556, -1.671909307631893],
[0.962914279874254,	-0.102326586625353, 2.857083360096907],
[0.317957619634043,	2.646768968413408, 1.412132053672896],
[-2.825388342924982, -0.928563755928189, -0.505520471387560],
[-0.317955944853142, -2.646769840660271, -1.412131825293682],
[-0.979776174195320, -2.312003751825495, 1.671909138648006],
[-0.962916072888105, 0.102326392265998,	-2.857083272537599],
[-2.023340541398004, 2.136128558801072,	-0.666071089291685],
[-2.033762834001679, 0.643989905095452, 2.132999911364582],
[0.000002325340981,	0.000000762100600, 0.000000414930733]]

#convert to Bohr
AtoBohr = 1.8897259886
pos_ne13 = pos_ne13 * AtoBohr

length(pos_ne13) == n_atoms || error("number of atoms and positions not the same - check starting config")

#boundary conditions 
bc_ne13 = SphericalBC(radius=6.4*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_ne13, bc_ne13)

#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot) for i in 1:n_traj]

#results = Output(n_bin, max_displ_vec)
results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)


Output{Float64}(100, 0.0, 0.0, Float64[], Float64[], Float64[], Vector{Float64}[], Vector{Float64}[], Float64[], Float64[], Float64[], Float64[])

Wanting to test the use of splatting and varargs.

Two versions of equilibration_cycle! exist, one requires (mc_states,move_strat,mc_params,results,pot,ensemble), the other, only (pot) if the splatting works appropriately, this needs to be correctly reordered

In [6]:
#initaliser = (mc_states,move_strat,mc_params,results,pot,ensemble)
#mc_states,move_strat,ensemble,results,delta_en_hist,delta_r2,step,n_steps,a,v,r = equilibration_cycle!(initaliser...)

(MCState{Float64, 13, SphericalBC{Float64}}[MCState{Float64, 13, SphericalBC{Float64}}(5.0, 63154.997052619445, Config{13, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[5.339202509675499, 1.7547286276557148, 0.9552945639202041], [3.8235622875401982, -4.036693324695786, 1.2586922223528678], [3.8432525502537267, -1.2169635315645375, -4.030786193502644], [1.8515104475315411, 4.369051328639683, -3.1594504692142205], [1.8196441394724319, -0.19336921007065874, 5.399104677171738], [0.6008527770958447, 5.001668105430829, 2.668542641160762], [-5.339209779512827, -1.754731061649526, -0.9552951725503948], [-0.6008496122188508, -5.001669753738395, -2.668542209586626], [-1.851508499387977, -4.369053575565342, 3.1594501498809775], [-1.8196475277773039, 0.19336884278473446, -5.399104511708638], [-3.823559204867802, 4.036697652557049, -1.2586918476896083], [-3.8432544820617607, 1.2169644600549232, 4.030785366187147], [4.394257284152319e-6, 1.4401613097476533e-6, 7.841053896189476e-7]], Spherical

# Splatted Input

Below we import the equilibration cycle, which should control the input and method, rather than the run! cycle. Then we write the testing_run function which does exactly what ptmc_run currently does, only using the splatting feature for its input.

In [3]:
#(mc_states,move_strat,mc_params,results,pot,ensemble)
import ParallelTemperingMonteCarlo.MCRun.equilibration_cycle!

In [4]:
function testing_run(inputs ; save::Bool=true,save_dir = pwd())
    
    mc_states,mc_params,move_strat,ensemble,results,delta_en_hist,delta_r2,start_counter,n_steps,a,v,r = equilibration_cycle!(inputs...)

    save_states(mc_params,mc_states,0,save_dir,move_strat,ensemble)
    
    for i = start_counter:mc_params.mc_cycles
        @inbounds ptmc_cycle!(mc_states,results,move_strat, mc_params, pot, ensemble ,n_steps ,a ,v ,r,save,i,save_dir,delta_en_hist,delta_r2)
    end
    println("MC loop done.")
    #Evaluation

    results = finalise_results(mc_states,mc_params,results)
    return 
end

testing_run (generic function with 1 method)

First we define inputs as the usual ptmc_run parameters and use the testing run function

In [5]:
inputs = mc_states,move_strat,mc_params,results,pot,ensemble
testing_run(inputs ;save=true)

Total number of moves per MC cycle: 13



equilibration done


MC loop done.


[0.0003314950966941005, 0.00034538634749973595, 0.00035341589626475533, 0.00037525626500724935, 0.00039621496821793794, 0.0004180776166825922, 0.00044151928937153024, 0.0004712660888332857, 0.0004960459589113677, 0.0005302674870648441, 0.0005442125495614913, 0.0006102202101247229, 0.0007350218559412198, 0.000716911361883324, 0.000779285502917946, 0.0008618172691087228, 0.0009556495566343556, 0.0010065573372527638, 0.0011949006734038598, 0.0013226577702060685, 0.0018156691731912348, 0.002399510072973971, 0.0031146421329893374, 0.003587448928561224, 0.0036186712090859295, 0.003420216920363774, 0.002974529046307324, 0.0028530510054192958, 0.00257097711787202, 0.002481631051920394, 0.002421989382515339, 0.002383195104032239]


Having defined params.data separately to go up to 1000000 rather than 40000 trials, this uses the smaller input, splats it into equilibration (which has a definition to restart from checkpoints) and then continue from there.

In [6]:
input = (pot,pwd())
testing_run(input;save=true)

MC loop done.
[0.00032726404304543876, 0.0003435822712089686, 0.000349909387271222, 0.0003767197095279849, 0.0003905138886441785, 0.0004175911925974066, 0.000441855447478287, 0.00046708452433497, 0.0004959269336748618, 0.0005226306527120828, 0.0005411528260427966, 0.0006036407792738793, 0.0007097167546260539, 0.0007039131905290315, 0.0007706640052807072, 0.0008419631036418525, 0.0009238428283394032, 0.001015843721991389, 0.001217834993703604, 0.0013675451254052517, 0.0018636247926556797, 0.0025349759227718646, 0.003107520531370498, 0.0035596142371289137, 0.0036373352272529003, 0.0034421886577047133, 0.0030336010421535328, 0.0029304330653185874, 0.0026264346205791106, 0.0024703608637133127, 0.00241905712153797, 0.0023795770307800785]


After discussions regarding how to organise and initialise different ptmc runs, I'm aiming to make a complete ptmc_run function from scratch with two methods for initialisation. focussing on how to create a run script.

Part of this is designing a start_file from which to begin the simulation. By design this should be similar to reading one config with parameters added.

In [16]:
import ParallelTemperingMonteCarlo.ReadSave.read_config
import ParallelTemperingMonteCarlo.ReadSave.read_configs
import ParallelTemperingMonteCarlo.ReadSave.initialise_params
import ParallelTemperingMonteCarlo.ReadSave.read_input

In [33]:
function testreadconfig(configvec)
    positions =  []
    if configvec[1,2] == "SphericalBC{Float64}"
        boundarycondition = SphericalBC(radius = (configvec[1,3]))
    end

    for row in eachrow(configvec[3:end,:])
        coord_atom = SVector(row[1] ,row[2] ,row[3] )
        push!(positions,coord_atom)
    end

    config = Config(positions,boundarycondition)
    
    return config
end

testreadconfig (generic function with 2 methods)

In [10]:
readfile = open("save.data","r+")
contents = readdlm(readfile)

close(readfile)

oneconfig = contents[1:23,:]
step,configdata = read_input(oneconfig)

In [42]:
teststate1 = read_config(configdata,13,pot)
configvec = configdata[7:end-1,:]
testconfig = testreadconfig(configvec)

MCState{Float64, 13, SphericalBC{Float64}}(5.0, 63154.997052619445, Config{13, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[0.9750964453281551, -1.7675687510885627, 2.5559926946942015], [-8.767484634733561, -5.285044408232165, 2.1984905531658008], [-3.4238996031848052, -5.899301289990651, 4.875959644667052], [-4.964477481704373, -1.0386922954978293, 2.649430047897722], [-6.460134248717091, 3.5147383744819702, 5.383664798327921], [-7.782561997536705, -2.4151075739026115, 7.3554652996043535], [-1.9865391114355184, -0.14396946260904125, 7.3419811469370275], [-2.135603835832441, 0.2054890696440337, -2.3714127600751613], [-6.5212219750860205, 3.6933448447513912, -0.5977225409887024], [-10.689719736310261, 0.0864176326719609, 2.547964588002527], [-1.4407450132212807, 3.5203664200403124, 2.5629337244167516], [-3.501132986841371, -5.431500041682502, -0.8136369780644138], [-7.67392029548055, -1.7954312521975746, -2.5945574205993376]], SphericalBC{Float64}(146.27079421912055)), [0.0 107.4

In [51]:
testconfig.bc == teststate1.config.bc

true

In [55]:
function read_state(oneconfigvec, potential)
    counta = [oneconfigvec[5,2], oneconfigvec[5,3]]
    countv = [oneconfigvec[5,4], oneconfigvec[5,5]]
    countr = [oneconfigvec[5,6], oneconfigvec[5,7]]
    countx = [oneconfigvec[5,8], oneconfigvec[5,9]]

    config = testreadconfig(oneconfigvec[7:end-1,:])
    mcstate = MCState(oneconfigvec[2,2], oneconfigvec[2,3],config, potential ; max_displ=[oneconfigvec[4,2], oneconfigvec[4,3], oneconfigvec[4,4] ], count_atom=counta,count_vol=countv,count_rot=countr,count_exc=countx)

    mcstate.en_tot = oneconfigvec[3,2]
    push!(mcstate.ham,oneconfigvec[6,2])
    push!(mcstate.ham,oneconfigvec[6,3])
    return mcstate
end

read_state (generic function with 2 methods)

In [59]:
teststate = read_state(configdata,pot)

MCState{Float64, 13, SphericalBC{Float64}}(5.0, 63154.997052619445, Config{13, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[0.9750964453281551, -1.7675687510885627, 2.5559926946942015], [-8.767484634733561, -5.285044408232165, 2.1984905531658008], [-3.4238996031848052, -5.899301289990651, 4.875959644667052], [-4.964477481704373, -1.0386922954978293, 2.649430047897722], [-6.460134248717091, 3.5147383744819702, 5.383664798327921], [-7.782561997536705, -2.4151075739026115, 7.3554652996043535], [-1.9865391114355184, -0.14396946260904125, 7.3419811469370275], [-2.135603835832441, 0.2054890696440337, -2.3714127600751613], [-6.5212219750860205, 3.6933448447513912, -0.5977225409887024], [-10.689719736310261, 0.0864176326719609, 2.547964588002527], [-1.4407450132212807, 3.5203664200403124, 2.5629337244167516], [-3.501132986841371, -5.431500041682502, -0.8136369780644138], [-7.67392029548055, -1.7954312521975746, -2.5945574205993376]], SphericalBC{Float64}(146.27079421912055)), [0.0 107.4